# Project III: Classical Non-linear Models and Police use of force
This notebook uses the Police Public Contact Survey (PPCS) dataset: `ppcs_cc.csv`.

In [254]:
%load_ext autoreload
%autoreload 2

import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import probit as probit
import logit as logit
import estimation as est
from scipy.stats import norm
from scipy.stats import t

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load the data 

In [255]:
# Load the dataset
dat = pd.read_csv('ppcs_cc.csv')

# Inspect distribution of the target variable
print("\nDistribution of 'anyuseofforce_coded':")
print(dat['anyuseofforce_coded'].value_counts(normalize=True))

# Inspect value counts for categorical variables
categorical_vars = ["sblack", "shisp", "swhite", "sother", "smale", "omajblack", 
                    "omajhisp", "omajwhite", "omajother", "osplit", "inctype_lin", "sbehavior"]

for var in categorical_vars:
    print(f"\nValue Counts for {var}:")
    print(dat[var].value_counts())


Distribution of 'anyuseofforce_coded':
anyuseofforce_coded
0    0.994999
1    0.005001
Name: proportion, dtype: float64

Value Counts for sblack:
sblack
0    3379
1     420
Name: count, dtype: int64

Value Counts for shisp:
shisp
0    3413
1     386
Name: count, dtype: int64

Value Counts for swhite:
swhite
1    2808
0     991
Name: count, dtype: int64

Value Counts for sother:
sother
0    3614
1     185
Name: count, dtype: int64

Value Counts for smale:
smale
1    2012
0    1787
Name: count, dtype: int64

Value Counts for omajblack:
omajblack
0    3568
1     231
Name: count, dtype: int64

Value Counts for omajhisp:
omajhisp
0    3708
1      91
Name: count, dtype: int64

Value Counts for omajwhite:
omajwhite
1    3433
0     366
Name: count, dtype: int64

Value Counts for omajother:
omajother
0    3755
1      44
Name: count, dtype: int64

Value Counts for osplit:
osplit
0    3799
Name: count, dtype: int64

Value Counts for inctype_lin:
inctype_lin
2    3641
1     158
Name: count, dtype

In [256]:
# Declare labels    
y_lab = 'anyuseofforce_coded'
#x_lab = ['const', 'sblack', 'shisp', 'sother']
#x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sempl', 'sincome', 'spop', 'sage', 'sagesq']
#x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sempl', 'sincome', 'spop', 'sage', 'sagesq', 'daytime', 'inctype_lin', 'omajblack', 'omajhisp', 'omajother']
x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sempl', 'sincome', 'spop', 'sage', 'sagesq', 'daytime', 'inctype_lin', 'omajblack', 'omajhisp', 'omajother', 'sbehavior']

dat['sage'] = dat['sage'] / 10
dat['sagesq'] = dat.sage * dat.sage 

#['const', 'sblack', 'shisp', 'sother', 'smale', 'sage', 'sempl', 'sincome', 'spop', 'daytime', 'inctype_lin', 'omajblack', 'omajhisp', 'omajother', 'sbehavior', 'year']  # Dropped 'osplit' and year // Multikollinearitet mellem hhv. "omajblack", "omajhisp" "omajwhite" og "omajother" +  "sblack", "shisp", "swhite", "sother"

## Declare labels, her er sother og omajother taget ud, men I kan bare rette til white
#y_lab = 'anyuseofforce_coded'
#x_lab = ["const", "sblack", "shisp", "swhite",  # Drop 'sother' giver Multikollinearitet  sammen med "sblack", "shisp", "swhite"
#         "smale", "sage", "sempl", "sincome", 
#         "spop", "daytime", "inctype_lin", 
#         "omajblack", "omajhisp", "omajwhite",  # Drop 'omajother' giver Multikollinearitet sammen med "omajblack", "omajhisp", "omajwhite"
#         "sbehavior"] # dropped year and osplit (indeholder kun 0?)

# create extra variables 
N = dat.shape[0]
dat['const'] = np.ones((N,))

# Rebuild the dataset without 'osplit'
dat = dat[[y_lab] + x_lab].copy()

# Check for missing data
assert dat.notnull().all(axis=1).all(), 'Missing values detected. Clean your data!'

dat.tail(5)

,anyuseofforce_coded,const,sblack,shisp,sother
3794,0,1.0,0,0,0
3795,0,1.0,0,0,0
3796,0,1.0,0,0,0
3797,0,1.0,0,0,0
3798,0,1.0,0,0,0


In [257]:
# Extract y and X
y = dat[y_lab].values
x = dat[x_lab].values
K = x.shape[1]

In [258]:
count_violent_1 = (dat['anyuseofforce_coded'] == 1).sum()
print(f"Number of 1s in 'anyuseofforce_coded': {count_violent_1}")

Number of 1s in 'anyuseofforce_coded': 19


## Estimate using Probit

In [259]:
# Initialize starting values
theta0 = probit.starting_values(y, x)

# Display starting values
print("Starting values for theta:", theta0)

probit_results = est.estimate(probit.q, theta0, y, x, cov_type='Sandwich')

Starting values for theta: [0.00801282 0.00984432 0.03084728 0.00550069]
Optimization terminated successfully.
         Current function value: 0.030440
         Iterations: 37
         Function evaluations: 195
         Gradient evaluations: 39


In [260]:
probit_tab = est.print_table(x_lab, probit_results, title=f'Probit, y = {y_lab}')
probit_tab

Optimizer succeded after 37 iter. (195 func. evals.). Final criterion:  0.03044.
Probit, y = anyuseofforce_coded


,theta,se,t
const,-2.7262,0.1099,-24.8038
sblack,0.2759,0.2346,1.1759
shisp,0.5703,0.1951,2.9227
sother,0.1790,0.3637,0.4920


## Estimate using Logit

In [261]:
logit_results = est.estimate(logit.q, theta0, y, x, cov_type='Sandwich')

Optimization terminated successfully.
         Current function value: 0.030440
         Iterations: 51
         Function evaluations: 265
         Gradient evaluations: 53


In [262]:
logit_tab = est.print_table(x_lab, logit_results, title=f'Logit, y = {y_lab}')
logit_tab

Optimizer succeded after 51 iter. (265 func. evals.). Final criterion:  0.03044.
Logit, y = anyuseofforce_coded


,theta,se,t
const,-5.7413,0.3344,-17.1703
sblack,0.8058,0.6695,1.2036
shisp,1.5933,0.5301,3.0060
sother,0.5342,1.0493,0.5090


## Average partial effects

### Probit

In [263]:
# Estimating the average partial effects 
indices = [x_lab.index('sblack'), x_lab.index('shisp'), x_lab.index('sother')]  
labels = ['sblack', 'shispanic', 'sother'] 
probit.properties(x, probit_results['theta'],probit_results['cov'],print_out = True,se=True,indices=indices, labels = labels)

,Estimate,SE,t-value,p-value
sblack,0.004,0.004,0.922,0.357
shispanic,0.013,0.007,1.915,0.056
sother,0.002,0.005,0.405,0.686


### Logit

In [264]:
# Estimating the average partial effects 
indices = [x_lab.index('sblack'), x_lab.index('shisp'), x_lab.index('sother')]  
labels = ['sblack', 'shispanic', 'sother']  
logit.properties(x, logit_results['theta'],logit_results['cov'],print_out = True,se=True,indices=indices, labels = labels)

,Estimate,SE,t-value,p-value
sblack,0.004,0.004,0.919,0.358
shispanic,0.013,0.007,1.897,0.058
sother,0.002,0.005,0.409,0.683


## Partial Effects

#### Defining different fixed vectors

In [265]:
#means of the regressors
print(f"{np.mean(dat['sage']):.2f}")
print(f"{np.mean(dat['sagesq']):.2f}")
print(f"{np.mean(dat['sincome']):.2f}")
print(f"{np.mean(dat['spop']):.2f}")

KeyError: 'sage'

In [ ]:
###ORIGINAL VECTOR###
# Let us make a vector of the values we want to investigate
x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sage', 'sempl', 'sincome', 'spop', 'daytime', 'inctype_lin', 'omajblack', 'omajhisp', 'omajother', 'sbehavior','sagesq']

x_me = np.array([1, 0, 0, 0, 1, 4.10, 0, 2.16, 1.36,0,1,0,0,0, 0,19.42]).reshape(1, -1)
pd.DataFrame(x_me, columns=x_lab, index=['x_me'])


In [250]:
### BEHAVIOR = 1 ###
# Let us make a vector of the values we want to investigate
#x_me= np.array([1, 0, 0, 0, 1, 4.1, 0, 2.16, 1.36,0,1,0,0,0,1,19.42]).reshape(1, -1)
#pd.DataFrame(x_me, columns=x_lab, index=['x_behavior'])


In [251]:
###DAYTIME = 1###
# Let us make a vector of the values we want to investigate
#x_me = np.array([1, 0, 0, 0, 1, 4.1, 0, 2.16, 1.36,1,1,0,0,0,0,19.42]).reshape(1, -1)
#pd.DataFrame(x_me, columns=x_lab, index=['x_daytime'])


## Swiching race from white to black, hispanic, other

In [ ]:
# We will look at the same values as previously, but we want to look at the difference for foreign = 0 and foreign = 1.
#k=1: black 
#k=2: hispanic
#k=3: other

k = 1
x_me2 = x_me.copy()
x_me2[:, k] = 1  # Keep everythin the same, but change foreign to 1 for all obs. 
pd.DataFrame(x_me2, columns=x_lab, index=['x_me2'])

### Probit

In [ ]:
b_pr = probit_tab.theta.values

me_black_pr = probit.G(x_me2@b_pr) - probit.G(x_me@b_pr) 

In [229]:
gx0 = norm.pdf(x_me@b_pr)
gx2 = norm.pdf(x_me2@b_pr)

grad_d_pr = gx2*x_me2 - gx0*x_me

In [230]:
def get_se(grad, cov):
    cov_me = grad@cov@grad.T
    return np.sqrt(np.diag(cov_me))

se_d_pr = get_se(grad_d_pr, probit_results['cov'])

In [ ]:
me_dict = {'Marginal Effect': me_black_pr[0],
           's.e.':            se_d_pr}
tab = pd.DataFrame(me_dict)
tab['t'] = tab['Marginal Effect'] / tab['s.e.']
tab.index.name = 'Var'
tab.round(6)

### Logit

In [232]:
b_lg = logit_tab.theta.values
me_black_lg = logit.G(x_me2@b_lg) - logit.G(x_me@b_lg)

In [233]:
#GAMMEL STD ERROR LØSNING

#gx0 = (np.exp(np.dot(x_me, b_lg))) / ((1 + np.exp(np.dot(x_me, b_lg)))**2)
#gx0 = (np.exp(np.dot(x_me2, b_lg))) / ((1 + np.exp(np.dot(x_me2, b_lg)))**2)

#grad_d_lg = gx2*x_me2 - gx0*x_me

#def get_se(grad, cov):
 #   cov_me = grad@cov@grad.T
 #   return np.sqrt(np.diag(cov_me))


#se_d_lg = get_se(grad_d_lg, logit_results['cov'])

In [234]:
def black_lg(b_lg):
    black_logit = logit.G(x_me2@b_lg) - logit.G(x_me@b_lg)
    return black_logit

qq = lambda b_lg: black_lg(b_lg)

grad_d_lg = est.centered_grad(qq,b_lg)

def get_se(grad, cov):
    cov_me = grad@cov@grad.T
    return np.sqrt(np.diag(cov_me))

se_d_lg = get_se(grad_d_lg, logit_results['cov'])

In [ ]:
me_dict = {'Marginal Effect': me_black_lg[0],
           's.e.':            se_d_lg}
tab = pd.DataFrame(me_dict)
tab['t'] = tab['Marginal Effect'] / tab['s.e.']
tab.index.name = 'Var'
tab.round(6)

In [236]:
#p_values = 2 * t.sf(np.abs(me_black_pr[0]/se_d_pr), df=x.shape[0] - x.shape[1]).round(4)
#print(p_values)